In [1]:
import pyensae
blobstorage = "hdblobstorage"
blobpassword = "jQIPVO/T54w8X49UPIbzAVvaNO3wmuUwI4/o9AJnCaPTHoCQnsaGBUkT4eIyi0BRQavgc/TAQMQwy8eu19CSBQ=="
hadoop_server = "clusterensaeazure1"
hadoop_password = "2azureENSAE;"
username = "test"

In [2]:
hadoop_server = "sparkclus2ensae"
hadoop_password = "ENSAEspark1;"
username = "test"

In [3]:
client, bs = %hd_open
client, bs

(<pyensae.remote.azure_connection.AzureClient at 0x96f5b38>,
 <azure.storage.blob.blobservice.BlobService at 0x48a8860>)

In [2]:
with open("yoyo.txt", "w", encoding="utf8") as f:
    for i in range(0,100000):
        f.write("{0},{1},{2}\n".format(i,i*2/5, -i+1))
    f.write("1,1,1")

In [3]:
with open("yoyow.txt", "w", encoding="utf8") as f:
    for i in range(0,100000):
        f.write("{0},{1},{2}\n".format(i/100,i, -i-5))
    f.write("1,1,1")

In [39]:
%tail yoyow.txt

In [40]:
%blob_up yoyow.txt hdblobstorage/test/yoyow.txt

'test/yoyow.txt'

In [106]:
%%PIG sample_explore.pig

ensemble = LOAD '$CONTAINER/$PSEUDO/yo.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
ensemble2 = LOAD '$CONTAINER/$PSEUDO/yo.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
test = GROUP ensemble BY $0 ;
testfilter = FILTER ensemble BY x <10;
testfilter2 = GROUP testfilter all;
testTotal= GROUP ensemble2 all;
somme = FOREACH testfilter2 {
    a= SUM(testfilter.x);
    b= group;
    c= a+1;
    GENERATE a,b,c;
    }
DUMP somme;
DUMP testfilter2;

--counting = FOREACH test GENERATE COUNT($1);
--DUMP counting;
--DESCRIBE ensemble;
--ens_group = GROUP ensemble ALL;
--DESCRIBE ens_group;
--sampled = FOREACH ens_group GENERATE FLATTEN(ensemble);
--DESCRIBE sampled;
--ens10 = LIMIT ensemble 10;
--ens_group10 = LIMIT en_group10 ;
--DUMP ens10;
--DUMP ens_group10;

In [57]:
%%PIG yoyop.pig
(t1:tuple(t1a:int, t1b:int,t1c:int),t2:tuple(t2a:int,t2b:int,t2c:int));

REGISTER '$CONTAINER/$PSEUDO/tupling.py' using jython as tupling;
data = LOAD '$CONTAINER/$PSEUDO/yoyo.txt' 
        USING PigStorage(',') AS (x1:double,x2:double,x3:double) ;
dataw = LOAD '$CONTAINER/$PSEUDO/yoyow.txt' 
        USING PigStorage(',') AS (w1:double,w2:double,w3:double) ;
--testing1 = foreach data generate TOTUPLE(x2, x1)as test;
testing = foreach data generate tupling.test(dataw, x1)as test;
DUMP testing;
--DESCRIBE testing;
--todisplay2 = FOREACH testing  GENERATE $1 AS test2;
--dump todisplay2;

In [173]:
%blob_close

True

In [60]:
%blob_up test_c.py hdblobstorage/test/test_c.py

'test/test_c.py'

In [58]:
jid = %hd_pig_submit yoyop.pig
jid

{'id': 'job_1451961118663_2275'}

In [60]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1451961118663_2275', None, 'done', True, 'SUCCEEDED')

In [55]:
%%PYTHON tupling.py

@outputSchema("y:bag{t:tuple(d:double,w:double)}")
def test(weights,d):
    bag =[]
    for i in range(len(weights)):
        bag.append((weights[i][1],d))
        
    return bag



In [54]:
bag=[]
bag.append((1,2,3))
bag.append((4,4,6))
print(len(bag))

2


In [56]:
%blob_up tupling.py hdblobstorage/test/tupling.py

'test/tupling.py'

In [64]:
%%PYTHON reservoir_sampling.py

import random

@outputSchemaFunction("rsSchema")
def reservoir_sampling(ensemble):
    ensemble = eval(ensemble)
    k = 10
    N = len(ensemble)
    echantillon = []
    for i, e in enumerate(ensemble):
        if len(echantillon) < k:
            echantillon.append(e)
        else:
            j = random.randint(0, i)
            if j < k:
                echantillon[j] = e
    return echantillon

@schemaFunction("rsSchema")
def rsSchema(input):
    return input

In [2]:
%%PIG sample_explore_complete.pig
REGISTER '$CONTAINER/$SCRIPTPIG/reservoir_sampling.py' using jython as myrs;
ensemble = LOAD '$CONTAINER/$PSEUDO/yo.txt'
        USING PigStorage('\t') AS (x:int, v:chararray) ;
DESCRIBE ensemble;
ens_group = GROUP ensemble ALL;
DESCRIBE ens_group;
sampled = FOREACH ens_group GENERATE FLATTEN(myrs.reservoir_sample(ensemble));
DESCRIBE sampled;
STORE sampled INTO
INTO '$CONTAINER/$PSEUDO/sampling/sample_rs.txt' USING PigStorage();

In [22]:
jid = %hd_pig_submit sample_explore_complete.pig -d reservoir_sampling.py
jid

AttributeError: 'list' object has no attribute 'endswith'

In [59]:
%%PYTHON test_c.py

from pig_util import outputSchema

@outputSchema("output_field_name:chararray")
def top_5_bigrams(tweets):

    
    return "testinnnng"

In [49]:
%%PIG exptestnumpy.pig

REGISTER '$CONTAINER/$PSEUDO/test_c.py' USING streaming_python AS nltk_udfs;
 
tweets =  LOAD '$CONTAINER/$PSEUDO/yo.txt'
        USING PigStorage('\t') AS (x:int, v:chararray) ;

bigrams_by_place = FOREACH tweets GENERATE  nltk_udfs.top_5_bigrams(v);
DUMP bigrams_by_place;

In [61]:
jid = %hd_pig_submit exptestnumpy.pig
jid

{'id': 'job_1451961118663_0134'}

In [64]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1451961118663_0134', '100% complete', 'done', True, 'SUCCEEDED')

In [23]:
import numpy as np
a=array([[12,3,4,5],[4,9,-1,5]],[[-1,-1,-1,6],[98,5,0,-8]],[[-1,-1,-1,6],[98,5,0,-8]])

NameError: name 'array' is not defined

In [28]:
a = np.ndarray(shape=(4,4,3))
a[1,]=[12,3,4,5]

ValueError: could not broadcast input array from shape (4) into shape (4,3)

In [25]:
np.gradient(a)

[array([[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]]), array([[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]]), array([[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]])]

In [61]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix as cm

pathToDataset = "datarating.csv"
data = pandas.read_csv(pathToDataset, sep=",",encoding="utf8")
data.head(10)
#adjlist = A.unstack([col_name, val_name], 'r_i')


,UserID,MovieID,Rating
0,1,661,3
1,1,914,3
2,1,3408,4
3,1,2355,5
4,1,1197,3
5,1,1287,5
6,1,2804,5
7,1,594,4
8,1,919,4
9,1,595,5


In [70]:
(row_name, col_name, val_name) = data.columns


In [101]:
a=data.pivot(index='UserID', columns='MovieID', values='Rating') 

In [103]:
a[np.isnan(a)]=0

In [105]:
a.shape

(6040, 3706)

In [ ]:

cData =  LOAD '$CONTAINER/imdd/ratings_mean_short.csv'
        using PigStorage (',')
        AS (UserID:int, MovieID:int, Rating:double) ;

-- On calcul le gamma
users = GROUP cData all ;
total= FOREACH users GENERATE MAX($1.UserID) as m, MAX($1.MovieID) as n;
sqrt_gamma = FOREACH total GENERATE SQRT(4*LOG(n)/0.1) as a;
        
        
cData_grp = GROUP cData BY MovieID;
-- DESCRIBE cData_grp;
-- On calcule la norme et le gamma sur la norme
Matrix_Norms = FOREACH cData_grp {
    tmp_out = FOREACH cData GENERATE Rating*Rating;
    out = SUM(tmp_out);
    GENERATE group as MovieID, SQRT(out) as Norm, (sqrt_gamma.$0/SQRT(out)>1?1:sqrt_gamma.$0/SQRT(out)) as Prob_j,RANDOM() as Rand_j,RANDOM() as Rand_k;
}


-- On ajoute la colonne Norm et probabilite dans cData
C = JOIN cData BY MovieID,Matrix_Norms BY MovieID;
D = FOREACH C GENERATE cData::UserID as UserID_f,cData::MovieID as MovieID_f,cData::Rating as Rating_f,
                        Matrix_Norms::Norm as Norm_f,Matrix_Norms::Prob_j as Prob_j_f,Rand_j as Rand_j,Rand_k as Rand_k;


Matrix_data = GROUP D BY UserID_f;
FF = FOREACH Matrix_data GENERATE group, FLATTEN(D.MovieID_f) as MV1;
-- Ajout des informations de MV1
FFF = JOIN FF BY MV1, D BY MovieID_f; 
-- Condition de validite IF
FFD = FFF ; -- FILTER FFF BY Rand_j<Prob_j_f;
-- Ajout de la seconde loop On peut peut etre appliquer un filtre avant le merge
GG = JOIN FFD BY group, D BY UserID_f;

DESCRIBE FFD;
DESCRIBE GG;

GGG = FOREACH GG GENERATE FFD::FF::group as UserID,FFD::FF::MV1 as MV_1,FFD::D::Rating_f as Rating_1,FFD::D::Norm_f as Norm_1,
                FFD::D::Prob_j_f as Proba_1,D::MovieID_f as MV_2,D::Rating_f as Rating_2,
                D::Norm_f as Norm_2,D::Prob_j_f as Proba_2,
                D::Rand_j as Rand_j,D::Rand_k as Rand_k;
                
                

-- mettre le calculs ici
-- HH = FOREACH GGG{
--    GENERATE Rating_1*Rating_2;
--}

-- DESCRIBE GG;



STORE GGG INTO '$CONTAINER/$PSEUDO/dom/matrix_all.txt' USING PigStorage(',');
-- STORE HH INTO '$CONTAINER/$PSEUDO/dom/matrix_val.txt' USING PigStorage(',');


In [4]:
%%PIG_azure dom.pig
cData =  LOAD '$CONTAINER/imdd/exp_original_short.csv'
        using PigStorage (',')
        AS (UserID:int,MovieID:int, Rating:double) ;

cData_grp = GROUP cData BY MovieID;

-- On calcule la norme sur les colonnes
Matrix_Norms = FOREACH cData_grp {
    tmp_out = FOREACH cData GENERATE Rating*Rating;
    out = SUM(tmp_out.$0);
    GENERATE group, out, SQRT(out)as norm;
}


-- On calcule le gamma
users = GROUP cData all ;
total= FOREACH users GENERATE MAX($1.UserID) as m, MAX($1.MovieID) as n;
DUMP total;
sqrt_gamma = FOREACH total GENERATE SQRT(4*LOG(n)/0.5) as a;


-- Association du MovieId (colonne de la matrice adjacente) avec sa norme 
inv_matrix_norms_gamma = FOREACH Matrix_Norms GENERATE group as MovieID, norm, ((sqrt_gamma.$0/norm)>1?1:sqrt_gamma.$0/norm) as ratio, RANDOM() as rand;

inv_matrix_norms_gamma2 = FOREACH Matrix_Norms GENERATE group as MovieID, norm, ((sqrt_gamma.$0/norm)>1?1:sqrt_gamma.$0/norm) as ratio, RANDOM() as rand;

-- Pour la premiere boucle
withinvgamma1 = JOIN cData BY MovieID , inv_matrix_norms_gamma BY MovieID;
-- Pour la deuxieme boucle
withinvgamma2 = JOIN cData BY MovieID , inv_matrix_norms_gamma2 BY MovieID;
second_loop = JOIN withinvgamma1 BY UserID ,withinvgamma2 By UserID;
DESCRIBE second_loop;

loops_clean = FOREACH second_loop GENERATE withinvgamma1::cData::UserID as UserID,withinvgamma1::cData::MovieID as MovieID1,
                                        withinvgamma1::cData::Rating as Rating1,
                                        withinvgamma1::inv_matrix_norms_gamma::ratio as ratio1,
                                        withinvgamma1::inv_matrix_norms_gamma::rand as rand1,
                                        withinvgamma1::inv_matrix_norms_gamma::norm as norm1,
                                        withinvgamma2::cData::MovieID as MovieID2,withinvgamma2::cData::Rating as Rating2,
                                        withinvgamma2::inv_matrix_norms_gamma2::ratio as ratio2,
                                        withinvgamma2::inv_matrix_norms_gamma2::rand as rand2,
                                        withinvgamma2::inv_matrix_norms_gamma2::norm as norm2;

filtered = FILTER loops_clean BY (ratio1> rand1) AND (ratio2 > rand2);
result = FOREACH filtered GENERATE TOMAP(CONCAT(((chararray)MovieID1),'_',((chararray)MovieID2)),  
                                (Rating1*Rating2 /((sqrt_gamma.$0>norm1?norm1:sqrt_gamma.$0)*(sqrt_gamma.$0>norm2?norm2:sqrt_gamma.$0))));

DUMP result;
STORE result INTO '$CONTAINER/$PSEUDO/dom/results_map2.txt' USING PigStorage(',');

In [5]:
jid = %hd_pig_submit dom.pig
jid

{'id': 'job_1452664005967_0685'}

In [364]:
jid = %hd_pig_submit exptestnumpy.pig
jid

{'id': 'job_1452664005967_0618'}

In [8]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1452664005967_0685', '100% complete', 'done', True, 'SUCCEEDED')

In [306]:
--second_loop_clean = FOREACH second_loop GENERATE withinvgamma::cData::UserID as UserID,
--                    withinvgamma::cData::MovieID as MovieID1,withinvgamma::cData::Rating as Rating1,
--                                                                                withinvgamma::inv_matrix_norms_gamma::ratio as Ratio1,withinvgamma::inv_matrix_norms_gamma::rand as Rand1,
                                                                                                                                    cData::MovieID: int,cData::Rating: double;

--cData_byuserid = GROUP withinvgamma BY UserID;
--DESCRIBE cData_byuserid;
--DUMP cData_byuserid;

-- cData_byuserid: {group: int,withinvgamma: {(cData::UserID: int,cData::MovieID: int,cData::Rating: double,inv_matrix_norms_gamma::MovieID: int,inv_matrix_norms_gamma::ratio: double)}}
--byuserid_raw = FOREACH cData_byuserid GENERATE group,FLATTEN($1);
--DESCRIBE byuserid_raw; 


--byuserid_clean1 = FOREACH byuserid_raw GENERATE group,withinvgamma::cData::UserID as UserID,withinvgamma::cData::MovieID as MovieID,withinvgamma::cData::Rating as Rating,withinvgamma::inv_matrix_norms_gamma::ratio as ratio;
--DESCRIBE byuserid_clean1;
--DUMP byuserid_clean;
--r_i = FOREACH cData_byuserid {
--    filtered = FOREACH $1 GENERATE *,((inv_matrix_norms_gamma::ratio) > RANDOM()?1:0) as filter_result;
--    second_loop = FILTER filtered BY filter_result ==1;
--    GENERATE group, second_loop;
--}
--r_i10 = LIMIT r_i 10;
--DUMP r_i10;
--filtered_k = FILTER inv_matrix_norms_gamma BY ratio > RANDOM();
--secondloop = JOIN cData BY MovieID , filtered_k BY MovieID;
--DUMP secondloop;

SyntaxError: invalid syntax (<ipython-input-306-6350b479f5e4>, line 1)

In [70]:
jid={'id': 'job_1452664005967_0312'}

In [27]:
import datetime 
res = client.job_queue()
res.reverse()   # les derniers jobs d'abord    
date = datetime.datetime.today()
for i, r in enumerate(res[:20]):
    st = client.job_status(r["id"])
    if st["status"]["state"] != "SUCCEEDED" :
        print(i, r, st["status"]["state"],date.fromtimestamp(float(st["status"]["startTime"])/1000), st["status"]["jobName"])
        print(st["userargs"].get("file", None), st["profile"].get("jobName", None))
        

2 {'id': 'job_1452664005967_0799', 'detail': None} RUNNING 2016-01-30 19:11:29.696000 PigLatin:load_results.pig
None PigLatin:load_results.pig
4 {'id': 'job_1452664005967_0797', 'detail': None} RUNNING 2016-01-30 19:10:39.920000 TempletonControllerJob
wasb://hdblobstorage@hdblobstorage.blob.core.windows.net/test/scripts/pig/load_results.pig TempletonControllerJob
10 {'id': 'job_1452664005967_0790', 'detail': None} FAILED 2016-01-30 18:23:45.987999 PigLatin:load_results.pig
None PigLatin:load_results.pig
12 {'id': 'job_1452664005967_0788', 'detail': None} FAILED 2016-01-30 18:19:43.742000 PigLatin:load_results.pig
None PigLatin:load_results.pig
15 {'id': 'job_1452664005967_0785', 'detail': None} FAILED 2016-01-30 18:11:06.529999 PigLatin:load_results.pig
None PigLatin:load_results.pig
18 {'id': 'job_1452664005967_0782', 'detail': None} FAILED 2016-01-30 18:00:49.536000 PigLatin:load_results.pig
None PigLatin:load_results.pig


In [224]:
 client.job_kill("job_1452664005967_0449")

{'callback': None,
 'completed': None,
 'exitValue': None,
 'id': 'job_1452664005967_0449',
 'msg': None,
 'parentId': 'job_1452664005967_0448',
 'percentComplete': None,
 'profile': {'jobFile': '/user/admin/.staging/job_1452664005967_0449/job.xml',
  'jobID': {'id': 449, 'jtIdentifier': '1452664005967'},
  'jobId': 'job_1452664005967_0449',
  'jobName': 'PigLatin:dom.pig',
  'queueName': 'default',
  'url': 'http://headnodehost:9014/cluster/app/application_1452664005967_0449',
  'user': 'admin'},
 'status': {'cleanupProgress': 0.0,
  'failureInfo': 'NA',
  'finishTime': 1453990272799,
  'historyFile': '',
  'jobACLs': {},
  'jobComplete': True,
  'jobFile': '/user/admin/.staging/job_1452664005967_0449/job.xml',
  'jobID': {'id': 449, 'jtIdentifier': '1452664005967'},
  'jobId': 'job_1452664005967_0449',
  'jobName': 'PigLatin:dom.pig',
  'jobPriority': 'NORMAL',
  'mapProgress': 0.0,
  'neededMem': 0,
  'numReservedSlots': 0,
  'numUsedSlots': 0,
  'priority': 'NORMAL',
  'queue': 'de

In [291]:
emit ={}
emit['1_2']=2
emit['1_3']=3
emit['1_4']=2
emit['2_1']=2
emit['2_2']=5
emit['2_3']=1
emit['2_4']=3

In [297]:
import pandas as pd
import numpy as np
np.stack(emit)


array(['2_4', '2_2', '2_1', '2_3', '1_4', '1_3', '1_2'], 
      dtype='<U3')

In [362]:
%%PIG_azure exptestnumpy.pig

DEFINE computeMatrixNorms(cData,sqrt_gamma) RETURNS Matrix_Norms {
    cData_grp = GROUP $cData BY MovieID;
    -- On calcule la norme et le gamma sur la norme
    $Matrix_Norms = FOREACH cData_grp {
        tmp_out = FOREACH $cData GENERATE Rating*Rating;
        out = SUM(tmp_out);
        GENERATE group as MovieID, SQRT(out) as Norm, ($sqrt_gamma.$0/SQRT(out)>1?1:$sqrt_gamma.$0/SQRT(out)) as Prob_j,RANDOM() as Rand_j,RANDOM() as Rand_k;
    }
}

cData =  LOAD '$CONTAINER/imdd/ratings_mean_short.csv'
        using PigStorage (',')
        AS (UserID:int, MovieID:int, Rating:double) ;

-- On calcul le gamma
users = GROUP cData all ;
total= FOREACH users GENERATE MAX($1.UserID) as m, MAX($1.MovieID) as n;
sqrt_gamma = FOREACH total GENERATE SQRT(4*LOG(n)/0.5) as a;
        
        
-- On calcule la norme et le gamma sur la norme
Matrix_Norms = computeMatrixNorms(cData,sqrt_gamma);

-- On ajoute la colonne Norm et probabilite dans cData
C = JOIN cData BY MovieID,Matrix_Norms BY MovieID;
D = FOREACH C GENERATE cData::UserID as UserID_f,cData::MovieID as MovieID_f,cData::Rating as Rating_f,
                        Matrix_Norms::Norm as Norm_f,Matrix_Norms::Prob_j as Prob_j_f,Rand_j as Rand_j,Rand_k as Rand_k;

Matrix_data = GROUP D BY UserID_f;
FF = FOREACH Matrix_data GENERATE group as UID, FLATTEN(D.MovieID_f) as MV1;
-- Ajout des informations de MV1
FFF = JOIN FF BY (UID,MV1), D BY (UserID_f,MovieID_f); 
-- Condition de validite premier IF
FFD = FILTER FFF BY RANDOM()<Prob_j_f;
-- Ajout de la seconde loop
GG = JOIN FFD BY UID, D BY UserID_f;
-- Cleaning du tableau
GGG = FOREACH GG GENERATE FFD::FF::UID as UserID,FFD::FF::MV1 as MV_1,FFD::D::Rating_f as Rating_1,FFD::D::Norm_f as Norm_1,
                FFD::D::Prob_j_f as Proba_1,D::MovieID_f as MV_2,D::Rating_f as Rating_2,
                D::Norm_f as Norm_2,D::Prob_j_f as Proba_2,
                D::Rand_j as Rand_j,D::Rand_k as Rand_k;
                                
-- Ajout de la deuxieme boucle
-- Condition de validite second IF
GGD = FILTER GGG BY RANDOM()<Proba_2;
DESCRIBE GGD;
-- Generation des similarites
HH = FOREACH GGD{
    val = Rating_1*Rating_2/(((sqrt_gamma.$0>Norm_1)?Norm_1:sqrt_gamma.$0)*((sqrt_gamma.$0>Norm_2)?Norm_2:sqrt_gamma.$0));
    GENERATE MV_1,MV_2,val;
}

STORE HH INTO '$CONTAINER/$PSEUDO/dom/matrix_all2.txt' USING PigStorage(',');


In [128]:
%%PIG_azure load_results.pig


cData =  LOAD '$CONTAINER/$PSEUDO/dom/matrix_all2.txt'
        using PigStorage (',')
        AS (MovieID1:int, MovieID2:int, sim:double) ;

filtered = FILTER cData BY MovieID1 == $MvID ;
ordered = ORDER filtered BY sim DESC;
ordered_limit = LIMIT ordered $size;
movies = FOREACH ordered_limit GENERATE MovieID2;
STORE movies INTO '$CONTAINER/imdd/dom/recom.txt' USING PigStorage(',');

In [137]:
if client.exists(bs, client.account_name, "$PSEUDO/imdd/dom/recom.txt"):
    r = client.delete_folder (bs, client.account_name, "$PSEUDO/imdd/dom/recom.txt")
jid = client.pig_submit(bs, blobstorage, "load_results.pig",params={"MvID":'1610',"size":"20"})
jid

{'id': 'job_1452664005967_0835'}

In [141]:
st = %hd_job_status jid["id"]
(st["id"],st["percentComplete"],st["completed"],
st["status"]["jobComplete"],st["status"]["state"])

('job_1452664005967_0835', '75% complete', 'done', True, 'SUCCEEDED')

In [142]:
import os
if os.path.exists("recom.txt"):os.remove("recom.txt")
%blob_downmerge /imdd/dom/recom.txt recom.txt

'recom.txt'

In [143]:
 with open('recom.txt', 'r') as f:
    read_data = f.read()
    print(read_data)

3255
3147
380
3095
3071
3068
3035
3030
2943
982
2858
368
920
356
349
318
2571
2501
2396
2353

